In [ ]:
import pandas as pd
import rioxarray
import xarray as xr

In [ ]:
df = pd.read_parquet(snakemake.input.changedbiaswind)

In [ ]:
df

In [ ]:
mean = df.groupby(["technology", "spatial"]).mean()
mean

In [ ]:
xar = (
    mean
    # .iloc[:10, :]
    .reset_index()
    .rename(columns={"spatial": "grid_cell", "0": "res"})
    .assign(
        x1_coord=lambda df: (
            df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[:-2]
        ),
        x2_coord=lambda df: (
            df.grid_cell.str.split("x").str[1].str.split("y").str[0].str[-2:]
        ),
        x=lambda df: df.x1_coord + "." + df.x2_coord,
        y1_coord=lambda df: df.grid_cell.str.split("y").str[1].str[:-2],
        y2_coord=lambda df: df.grid_cell.str.split("y").str[1].str[-2:],
        y=lambda df: df.y1_coord + "." + df.y2_coord,
    )
    .astype({"x": float, "y": float})
    .loc[:, ["x", "y", "res"]]
    .set_index(["y", "x"])
    .to_xarray()
    .fillna(0)
)
xar

In [ ]:
smallestincluded = float(snakemake.wildcards.cutoff_wind)

In [ ]:
smallestincluded

In [ ]:
excluded = xar.where(xar >= smallestincluded, other=1)
excluded = excluded.where(xar < smallestincluded, other=0)

In [ ]:
(
    excluded
    .res
    .to_pandas()
    .stack()
    .to_frame()
    .reset_index()
    .assign(band=1)
    .set_index(["band","y","x"])
    .rename(columns={0:"band_data"})
    .to_xarray()
    .band_data
    .rio.write_crs("epsg:4326", inplace=True)
    .rio.to_raster(snakemake.output.exclusiontiff)
)

In [ ]:
#(
#    excluded.res.transpose("y", "x")
#    .rio.write_crs("epsg:4326", inplace=True)
#    .rio.to_raster(snakemake.output.exclusiontiff)
#)